In [ ]:
import dash
import more_itertools
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px


# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')


# Initialize the Dash app
app = dash.Dash(__name__)

# TASK 2.1: Create a Dash application and give it a meaningful title
# Set the title of the dashboard
#app.title = "Automobile Statistics Dashboard"
# Build dash app layout
app.layout = html.Div(children=[ html.H1('Automobile Sales Statistics Dashboard', 
                                style={'textAlign': 'left', 'color': '#503D36',
                                'font-size': 24}),
                                
# TASK 2.2: Add drop-down menus to your dashboard with appropriate titles and options                               


                                  dcc.Dropdown(id='dropdown-statistics', 
                   options=[
                           {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                           {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                           ],
                  placeholder= {'label':'Select a report type', 'value': 'Select Statistics'}
                  style={'width': '80%', 'padding': '3px', 'font-size': 30, 'text-align-last': 'center'})
# Create SECOND dropdown menu for selecting the YEAR options to it
# Créer un DEUXIÈME menu déroulant pour sélectionner les options ANNÉE
dcc.Dropdown(id='select-year', 
                   options=[{'label': i, 'value': i} for i in year_list= [i for i in range(1980, 2024, 1)]],
                  placeholder= {'label':'Select a year', 'value': 'Select year'},
                  style={'width': '80%', 'padding': '3px', 'font-size': 30, 'text-align-last': 'center'})


# TASK 2.3: Add a division for output display with appropriate id and classname property
                                
html.Div([
    html.Div(id='output-container', className='chart-grid', style={'display': 'flex'}),
])

# TASK 2.4: Creating Callbacks; Define the callback function to update the input container based on the selected statistics and the output container    
@app.callback(
    Output(component_id='select year', component_property='disabled'),
    Input(component_id='dropdown-statistics',component_property='disabled'))

def update_input_container(input_year, input_statistics):
    if input_year =='Yearly Statistics': 
        return False
    else: 
        return True                                

                                
# Callback for plotting
@app.callback(
    Output(component_id='dcc.Graph()', component_property='children'),
    [Input(component_id='select-year', component_property='disabled'), Input(component_id='dropdown-statistics', component_property='disabled')])
                                
def update_output_container(select-year, dropdown-statistics):
    if select-year == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]                                

# TASK 2.5: Create and display graphs for Recession Report Statistics
                                
#Plot 1 Automobile sales fluctuate over Recession Period (year wise) using line chart
         # grouping data for plotting
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        # Plotting the line graph
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                x='Year',
                y='Automobile_sales',
                title="Sales Trend Vehicle-wise during Recession"))

#Plot 2 Calculate the average number of vehicles sold by vehicle type and represent as a Bar chart
#Graphique 2 : Calculez le nombre moyen de véhicules vendus par type de véhicule et représentez-le sous forme de graphique à barres

automobile_Sales_by_type = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()                                
                                
# grouping data for plotting
       automobile_Sales_by_type = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
       # Plotting the line graph
       R_chart2 = dcc.Graph(
            figure=px.line(automobile_Sales_by_type , 
                x='Vehicle Type',
                y='Automobile sales',
                title='the average number of vehicles sold by type'.format(input_year)))
                                
                                
# Plot 3 : Pie chart for total expenditure share by vehicle type during recessions
# Diagramme circulaire pour la part des dépenses totales par type de véhicule pendant les récessions
            # grouping data for plotting
            exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
            R_chart3 = dcc.Graph(
                    figure=px.pie(exp_rec,
                    values='Advertising_Expenditure',
                 names='Vehicle_Type',
                 title="total expenditure share by vehicle type during recessions"
                )
        )

# Plot 4 Develop a Bar chart for the effect of unemployment rate on vehicle type and sales
# Graphique 4 : Développer un graphique à barres pour l’effet du taux de chômage sur le type de véhicule et les ventes
unemp_data= recession_data.groupby(['Vehicle_Type', 'unemployment_rate'])['Automobile_Sales'].mean().reset_index()
R_chart4 = dcc.Graph(
              figure=px.bar(
                  unemp_data,
                  x='unemployment_rate',
                  y='Automobile_Sales',
                  color='Vehicle_Type',
                  labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
                  title="Effect of Unemployment Rate on Sales of various Vehicle Types")
          )



return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)]),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)])
            ]

               
                             
# TASK 2.6: Create and display graphs for Yearly Report Statistics


# Yearly Statistic Report Plots                             
    elif (input_year and selected_statistics=='Yearly Statistics') :
        yearly_data = data[data['Year'] == year_list]
                              

# Plot 1 :Yearly Automobile sales using line chart for the whole period.
# Graphique 1 : Ventes annuelles d’automobiles à l’aide d’un graphique linéaire pour toute la période.                                   

        yas= yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(
                yas,
                x='Year',
                y='Automobile_Sales',
                title='Yearly Automobile sales by year',
                hue='Year',
                style='Year',
                markers='o',
                err_style=None
            ))
            

# Plot 2 :Total Monthly Automobile sales using line chart.
# Graphique 2 : Ventes mensuelles totales d’automobiles à l’aide d’un graphique linéaire.                                   
       Mas= yearly_data.groupby('Monthly')['Automobile_Sales'].sum().reset_index()
       Y_chart2 = dcc.Graph(
            figure=px.line(
                Mas,
                x='Monthly',
                y='Automobile_Sales',
                title='Total Monthly Automobile sales',
                hue='Vehicle_Type',
                style='Vehicle_Type',
                markers='o',
                err_style=None
            ))                            


 
# Plot bar chart for average number of vehicles sold during the given year
# Graphique à barres pour le nombre moyen de véhicules vendus au cours de l’année donnée                                   

  
        avr_vdata=yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure= px.bar(
                avr_vdata,
                x= 'Year',
                y= 'Automobile_Sales',
                title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year)))


# Plot 4 Total Advertisement Expenditure for each vehicle using pie chart
# Graphique 4 Dépenses publicitaires totales pour chaque véhicule à l’aide d’un graphique circulaire
     ad_exP_by_vcle = yearly_data.groupby()('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
     Y_chart4 = dcc.Graph(
                    figure=px.pie(
                        ad_exP_by_vcle,
                    values='Advertising_Expenditure',
                 names='Vehicle_Type',
                 title="Total Advertisement Expenditure for each vehicle"
                )
        )                             

 return [

            html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': 'flex'})
            ]
                                
                                
else:
        return None

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)                                